# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Liu  ->  J. Liu  |  ['J. Liu']


K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Arxiv has 79 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.17440


extracting tarball to tmp_2503.17440...

 done.


Found 84 bibliographic references in tmp_2503.17440/paper.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.17455


...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.18791


extracting tarball to tmp_2503.18791...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2503.18791/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'Tables/galaxyfits_fixed' from 'tmp_2503.18791/Tables/galaxyfits_fixed.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'Tables/galaxysample.tab' from 'tmp_2503.18791/Tables/galaxysample.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python

Found 56 bibliographic references in tmp_2503.18791/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.17440-b31b1b.svg)](https://arxiv.org/abs/2503.17440) | **The resolved star-formation efficiency of early-type galaxies**  |
|| T. G. Williams, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-03-25*|
|*Comments*| *13 pages, 42 Figures (27 in Appendices). Accepted for publication in MNRAS*|
|**Abstract**|            Understanding how and why star formation varies between galaxies is fundamental to our comprehension of galaxy evolution. In particular, the star-formation efficiency (SFE; star-formation rate or SFR per unit cold gas mass) has been shown to vary substantially both across and within galaxies. Early-type galaxies (ETGs) constitute an extreme case, as about a quarter have detectable molecular gas reservoirs but little to no detectable star formation. In this work, we present a spatially-resolved view of the SFE in ten ETGs, combining state-of-the-art Atacama Large Millimeter/submillimeter Array (ALMA) and Multi Unit Spectroscopic Explorer (MUSE) observations. Optical spectroscopic line diagnostics are used to identify the ionized emission regions dominated by star-formation, and reject regions where the ionization arises primarily from other sources. We identify very few regions where the ionization is consistent with pure star formation. Using ${\rm H}\alpha$ as our SFR tracer, we find that previous integrated measurements of the star-formation rate based on UV and 22$\mu$m emission are systematically higher than the SFR measured from ${\rm H}\alpha$. However, for the small number of regions where ionization is primarily associated with star formation, the SFEs are around 0.4 dex higher than those measured in star-forming galaxies at a similar spatial resolution (with depletion times ranging from $10^8$ to $10^{10}$ yr). Whilst the SFE of ETGs is overall low, we find that the SFEs of individual regions within ETGs can be similar to, or higher than, similar sized regions within star-forming galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.17455-b31b1b.svg)](https://arxiv.org/abs/2503.17455) | **Euclid preparation LXX. Forecasting detection limits for intracluster light in the Euclid Wide Survey**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-25*|
|*Comments*| *21 pages, 13 figures, 2 tables. Accepted for publication in A&A*|
|**Abstract**|            The intracluster light (ICL) permeating galaxy clusters is a tracer of the cluster's assembly history, and potentially a tracer of their dark matter structure. In this work we explore the capability of the Euclid Wide Survey to detect ICL using H-band mock images. We simulate clusters across a range of redshifts (0.3-1.8) and halo masses ($10^{13.9}$-$10^{15.0}$ M$_\odot$), using an observationally motivated model of the ICL. We identify a 50-200 kpc circular annulus around the brightest cluster galaxy (BCG) in which the signal-to-noise ratio (S/N) of the ICL is maximised and use the S/N within this aperture as our figure of merit for ICL detection. We compare three state-of-the-art methods for ICL detection, and find that a method that performs simple aperture photometry after high-surface brightness source masking is able to detect ICL with minimal bias for clusters more massive than $10^{14.2}$ M$_\odot$. The S/N of the ICL detection is primarily limited by the redshift of the cluster, driven by cosmological dimming, rather than the mass of the cluster. Assuming the ICL in each cluster contains 15% of the stellar light, we forecast that Euclid will be able to measure the presence of ICL in up to $\sim80000$ clusters of $>10^{14.2}$ M$_\odot$ between $z=0.3$ and 1.5 with a S/N$>3$. Half of these clusters will reside below $z=0.75$ and the majority of those below $z=0.6$ will be detected with a S/N $>20$. A few thousand clusters at $1.3<z<1.5$ will have ICL detectable with a S/N greater than 3. The surface brightness profile of the ICL model is strongly dependent on both the mass of the cluster and the redshift at which it is observed so the outer ICL is best observed in the most massive clusters of $>10^{14.7}$ M$_\odot$. Euclid will detect the ICL at more than 500 kpc distance from the BCG, up to $z=0.7$, in several hundred of these massive clusters over its large survey volume.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.17921-b31b1b.svg)](https://arxiv.org/abs/2503.17921) | **Empirical SED Templates for Star Clusters Observed with HST and JWST: No Strong PAH or IR Dust Emission after Five Myr**  |
|| B. C. Whitmore, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-03-25*|
|*Comments*| *35 pages, 15 figures, 7 tables*|
|**Abstract**|            JWST observations, when combined with HST data, promise to improve age estimates of star clusters in nearby spiral galaxies. However, feedback from young cluster stars pushes out the natal gas and dust, making cluster formation and evolution a challenge to model. Here, we use JWST + HST observations of the nearby spiral galaxy NGC 628 to produce spectral energy distribution (SED) templates of compact star clusters spanning 275 nm through 21 {\mu}m. These preliminary SEDs capture the cluster stars and associated gas and dust within radii of 0.12" to 0.67" (corresponding to 6 to 33 pc at the distance of NGC 628). One important finding is that the SEDs of 1, 2, 3, and 4 Myr clusters can be differentiated in the infrared. Another is that in 80-90% of the cases we study, the PAH and H_alpha emission track one another, with the dust responsible for the 3.3 {\mu}m PAH emission largely removed by 4 Myr, consistent with pre-supernova stellar feedback acting quickly on the surrounding gas and dust. Nearly-embedded cluster candidates have infrared SEDs which are quite similar to optically visible 1 to 3 Myr clusters. In nearly all cases we find there is a young star cluster within a few tenths of an arcsec (10 - 30 pc) of the nearly embedded cluster, suggesting the formation of the cluster was triggered by its presence. The resulting age estimates from the empirical templates are compatible both with dynamical estimates based on CO superbubble expansion velocities, and the TODDLERS models which track spherical evolution of homogeneous gas clouds around young stellar clusters.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.18791-b31b1b.svg)](https://arxiv.org/abs/2503.18791) | **The PHANGS-HST-Halpha Survey: Warm Ionized Gas Physics at High Angular resolution in Nearby GalaxieS with the Hubble Space Telescope**  |
|| R. Chandar, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-03-25*|
|*Comments*| *published in the Astronomical Journal*|
|**Abstract**|            The PHANGS project is assembling a comprehensive, multi-wavelength dataset of nearby (~5-20 Mpc), massive star-forming galaxies to enable multi-phase, multi-scale investigations into the processes that drive star formation and galaxy evolution. To date, large survey programs have provided molecular gas (CO) cubes with ALMA, optical IFU spectroscopy with VLT/MUSE, high-resolution NUV--optical imaging in five broad-band filters with HST, and infrared imaging in NIRCAM+MIRI filters with JWST. Here, we present PHANGS-HST-Halpha, which has obtained high-resolution (~2-10 pc), narrow-band imaging in the F658N or F657N filters with the HST/WFC3 camera of the warm ionized gas in the first 19 nearby galaxies observed in common by all four of the PHANGS large programs. We summarize our data reduction process, with a detailed discussion of the production of flux-calibrated, Milky Way extinction corrected, continuum-subtracted Halpha maps. PHANGS-MUSE IFU spectroscopy data are used to background subtract the HST-Halpha maps, and to determine the [NII] correction factors for each galaxy. We describe our public data products and highlight a few key science cases enabled by the PHANGS-HST-Halpha observations.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.17732-b31b1b.svg)](https://arxiv.org/abs/2503.17732) | **Probing the He II reionization ERa via Absorbing C IV Historical Yield (HIERACHY) III: The C IV absorber catalog and initial results on cosmic abundance evolution at $z\approx 3-5$**  |
|| X. Yu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-03-25*|
|*Comments*| *26 pages, 12 figures, and 10 tables; Accepted for publication in APJ*|
|**Abstract**|            As part of the HIERACHY program, we collect the high-SN and high-spectral resolution optical spectra of 25 quasars at $z\approx4-5$ to constrain the C IV evolution at $z\approx 3-5$. In this paper, we report a catalog of 626 (1263) C IV absorption systems (components) at $z\approx3-5$ with a 50% completeness column density of log$(N_{\rm CIV}/\rm cm^{-2}) \approx 12.3$. The HIERACHY/MIKE C IV sample is one of the best C IV absorber samples optimized to study the IGM during the He II reionization epoch. Using 557 (1090) intervening absorption systems (components), we found the column density distribution function of C IV absorption systems with log$(N_{\rm CIV}/\rm cm^{-2})\gtrsim 12.3$ has a broken power-law shape, with the turn-over column density log$(N_{\rm crit}/\rm cm^{-2}) = 13.35^{+0.20}_{-0.19}$, which is close to or smaller than the detection limit of most literature samples. We also found that both comoving path length number density $dn/dX$ and cosmic abundance $\Omega$ for C IV absorption systems with log$(N_{\rm CIV}/\rm cm^{-2})> 13.2$ show an increase (at the 2.2$\sigma$ and 1.4$\sigma$ levels, respectively) from redshift $z\approx5$ to 3, while absorption systems with log$(N_{\rm CIV}/\rm cm^{-2})= 12.3-13.2$ exhibit a constant $dn/dX$ and $\Omega_{\rm CIV}$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.18145-b31b1b.svg)](https://arxiv.org/abs/2503.18145) | **Asymptotic states of fast neutrino-flavor conversions in the three-flavor framework**  |
|| <mark>J. Liu</mark>, et al. |
|*Appeared on*| *2025-03-25*|
|*Comments*| **|
|**Abstract**|            There has been growing evidence that mu and tau neutrinos are noticeably different due to the emergence of muons in core-collapse supernovae (CCSNe) and binary neutron star mergers (BNSMs). Recent theoretical studies also suggest that all flavors of neutrinos and antineutrinos inevitably experience some flavor mixing instabilities including fast neutrino flavor conversions (FFC), which corresponds to one of the collective neutrino oscillations powered by neutrino self-interactions. This represents a need for quantum kinetic treatment in the numerical modeling of neutrino dynamics, which is, however, a formidable computational challenge. In this paper, we present an approximate method to predict asymptotic states of FFC without solving a quantum kinetic equation under a three-flavor framework, in which mu and tau neutrino distributions are not necessarily identical to each other. The approximate method is developed based on a Bhatnagar-Gross-Krook (BGK) relaxation time prescription, capable of capturing essential features of mixing competitions among three different flavor-coherent states. Our proposed scheme is computationally inexpensive and easy to implement in any classical neutrino transport scheme.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.17440/./ngc4694_bpt.png', 'tmp_2503.17440/./ngc4694_whan.png', 'tmp_2503.17440/./ngc4694_sfr_map.png']
copying  tmp_2503.17440/./ngc4694_bpt.png to _build/html/
copying  tmp_2503.17440/./ngc4694_whan.png to _build/html/
copying  tmp_2503.17440/./ngc4694_sfr_map.png to _build/html/
exported in  _build/html/2503.17440.md
    + _build/html/tmp_2503.17440/./ngc4694_bpt.png
    + _build/html/tmp_2503.17440/./ngc4694_whan.png
    + _build/html/tmp_2503.17440/./ngc4694_sfr_map.png
found figures ['tmp_2503.17455/./grid_2panel_mass_fICL_50_200_sim.png', 'tmp_2503.17455/./ICL_fits_comparison_sigmaratio_ficl.png', 'tmp_2503.17455/./sn_curve_multi_newsys_new.png']
copying  tmp_2503.17455/./grid_2panel_mass_fICL_50_200_sim.png to _build/html/
copying  tmp_2503.17455/./ICL_fits_comparison_sigmaratio_ficl.png to _build/html/
copying  tmp_2503.17455/./sn_curve_multi_newsys_new.png to _build/html/
exported in  _build/html/2503.17455.md
    + _build/html/tmp_2503.17455/./grid_2p

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sigsfr}{\Sigma_{\rm SFR}}$
$\newcommand{\siggas}{\Sigma_{\rm gas}}$
$\newcommand{\ha}{{\rm H}\alpha}$
$\newcommand{\hb}{{\rm H}\beta}$
$\newcommand{\aco}{\alpha_\mathrm{CO}}$
$\newcommand{\ewha}{W_\mathrm{H\alpha}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The resolved star-formation efficiency of early-type galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.17440-b31b1b.svg)](https://arxiv.org/abs/2503.17440)<mark>Appeared on: 2025-03-25</mark> -  _13 pages, 42 Figures (27 in Appendices). Accepted for publication in MNRAS_

</div>
<div id="authors">

T. G. Williams, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Understanding how and why star formation varies between galaxies is fundamental to our comprehension of galaxy evolution. In particular, the star-formation efficiency (SFE; star-formation rate or SFR per unit cold gas mass) has been shown to vary substantially both across and within galaxies. Early-type galaxies (ETGs) constitute an extreme case, as about a quarter have detectable molecular gas reservoirs but little to no detectable star formation. In this work, we present a spatially-resolved view of the SFE in ten ETGs, combining state-of-the-art Atacama Large Millimeter/submillimeter Array (ALMA) and Multi Unit Spectroscopic Explorer (MUSE) observations. Optical spectroscopic line diagnostics are used to identify the ionized emission regions dominated by star-formation, and reject regions where the ionization arises primarily from other sources. We identify very few regions where the ionization is consistent with pure star formation. Using $\ha$ as our SFR tracer, we find that previous integrated measurements of the star-formation rate based on UV and 22 $\micron$ emission are systematically higher than the SFR measured from $\ha$ . However, for the small number of regions where ionization is primarily associated with star formation, the SFEs are around 0.4 dex higher than those measured in star-forming galaxies at a similar spatial resolution (with depletion times ranging from $10^8$ to $10^{10}$ yr). Whilst the SFE of ETGs is overall low, we find that the SFEs of individual regions within ETGs can be similar to, or higher than, similar sized regions within star-forming galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2503.17440/./ngc4694_bpt.png" alt="Fig6" width="100%"/>

**Figure 6. -** BPT diagram of individual spaxels within the galaxy NGC 4694. _ Left_: [O{\sc iii}] -- [N{\sc ii}] diagnostic. _ Right:_[O{\sc iii}] -- [S{\sc ii}] diagnostic. The demarcation from [ and Kauffmann (2003)](https://ui.adsabs.harvard.edu/abs/2003MNRAS.346.1055K) is shown as a solid black line, and those from [Kewley, et. al (2006)](https://ui.adsabs.harvard.edu/abs/2006MNRAS.372..961K) as black dashed and dot-dashed lines. Data points are coloured by their positions in the BPT diagram, and are grey if they do not meet our signal-to-noise thresholds. The error-bars are typically on the order of 0.01 for these ratios as the lines are strongly detected, so these uncertainties are negligible. (*fig:ngc4694_bpt*)

</div>
<div id="div_fig2">

<img src="tmp_2503.17440/./ngc4694_whan.png" alt="Fig12" width="100%"/>

**Figure 12. -** WHAN diagnostics of individual spaxels within the galaxy NGC 4694. _ Left_: Map of the WHAN classes overlaid on an $\ha$ map of the galaxy. _ Right_: WHAN classes defined using the $\ewha$ -- ([N{\sc ii}]/$\ha$ plane. We use the classes of [ and Mateus (2011)](https://ui.adsabs.harvard.edu/abs/2011MNRAS.413.1687C): star-forming (SF), strong AGN (sAGN), weak AGN (wAGN), retired (R) and passive (P). Here, we have dropped the `galaxy' nomenclature from the retired and passive galaxies ued in [ and Mateus (2011)](https://ui.adsabs.harvard.edu/abs/2011MNRAS.413.1687C), to highlight that our observations are resolved. (*fig:ngc4694_whan*)

</div>
<div id="div_fig3">

<img src="tmp_2503.17440/./ngc4694_sfr_map.png" alt="Fig8" width="100%"/>

**Figure 8. -** SFR (left) and BPT (right) maps of the galaxy NGC 4694. The pixels here correspond to a 150 pc-size region, and pixels are only coloured if they pass our S/N cuts, and the SFR maps are calculated only for spaxels that fall in the SF region of the BPT diagram. The greyscale shows $\ha$ emission in both cases. In the left panel, for the small number of spaxels classified as star-forming, we show the SFR surface densities in colour. The right panel shows spaxels coloured by their BPT classification, with the same colours as in Figure \ref{fig:ngc4694_bpt}. For this galaxy, the majority of the spaxels are defined as having their ionization dominated by processes other than star-formation. (*fig:ngc4694_sfr_map*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.17440"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Msun}{M_\sun}$
$\newcommand{\Mhalo}{M_\mathrm{h}}$
$\newcommand{\logMhalo}{\logten(\Mhalo/\Msun)}$
$\newcommand{\Mstar}{M_*}$
$\newcommand{\logMstar}{\logten(\Mstar/\Msun)}$
$\newcommand{\fICL}{f_\textrm{ICL}}$
$\newcommand{\Rmax}{R_\textrm{max}}$
$\newcommand{\SN}{\textrm{S/N}}$
$\newcommand{\ICLap}{(BCG+ICL)_{{50}-{200}}}$
$\newcommand{\nina}[1]{{\color{orange}\textsf{#1}}}$
$\newcommand{\eceb}[1]{{#1}}$
$\newcommand{\referee}[1]{#1}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$</div>



<div id="title">

# $\Euclid$ preparation: LXX. Forecasting detection limits for intracluster light in the Euclid Wide Survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.17455-b31b1b.svg)](https://arxiv.org/abs/2503.17455)<mark>Appeared on: 2025-03-25</mark> -  _21 pages, 13 figures, 2 tables. Accepted for publication in A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The intracluster light (ICL) permeating galaxy clusters is a tracer of the cluster's assembly history, and potentially a tracer of their dark matter structure.In this work we explore the capability of the Euclid Wide Survey to detect ICL using $\eceb{\HE-band}$ mock images. We simulate clusters across a range of redshifts (0.3--1.8) and halo masses ( $10^{13.9}$ -- $10^{15.0} \Msun$ ), using an observationally motivated model of the ICL. We identify a 50--200 kpc circular annulus around the brightest cluster galaxy (BCG) in which the signal-to-noise ratio (S/N) of the ICL is maximised and use the S/N within this aperture as our figure of merit for ICL detection. We compare three state-of-the-art methods for ICL detection, and find that a method that performs simple aperture photometry after high-surface brightness source masking is able to detect ICL with minimal bias for clusters more massive than $10^{14.2} \Msun$ .The S/N of the ICL detection is primarily limited by the redshift of the cluster, driven by cosmological dimming, rather than the mass of the cluster. Assuming the ICL in each cluster contains 15 \% of the stellar light, we forecast that $\Euclid$ will be able to measure the presence of ICL in $\eceb{up to}$ $\sim80$ 000 clusters of $>10^{14.2} \Msun$ between $z=0.3$ and 1.5 with a $\SN>3$ . Half of these clusters will reside below $z=0.75$ and the majority of those below $z=0.6$ will be detected with a $\SN>20$ . A few thousand clusters at $1.3<z<1.5$ will have ICL detectable with a S/N greater than 3.The surface brightness profile of the ICL model is strongly dependent on both the mass of the cluster and the redshift at which it is observed so the outer ICL is best observed in the most massive clusters of $>10^{14.7} \Msun$ . $\Euclid$ will detect the ICL at more than 500 kpc distance from the BCG, up to $z=0.7$ , in several hundred of these massive clusters over its large survey volume.

</div>

<div id="div_fig1">

<img src="tmp_2503.17455/./grid_2panel_mass_fICL_50_200_sim.png" alt="Fig8" width="100%"/>

**Figure 8. -** _Left_: \HE-band S/N of the ICL + BCG measured within a 50--200 kpc annulus, interpolated across halo mass and redshift space, using `smoothed' ICL parameters calculated as described in Sect. \ref{sec:fitting_sn} assuming a fixed ICL fraction of $15\%$. Coloured circles indicate the S/N values from the individual model clusters for comparison. Contours indicate the lines corresponding to a $\SN= 3, 10$ and 20 as labelled. Dashed contours indicate the threshold halo mass above which the total number of clusters in a redshift bin of $\Delta z=0.1$ is expected to be 10, 100, and 1000, across the footprint of the EWS, calculated as described in Sect. \ref{sec:haloestimates}. By the same estimates, the grey shaded region to the upper-right indicates the parameter space in which we expect to find no clusters according to the halo mass function.
_Right_: \HE-band S/N of the BCG+ICL measured within a 50--200 kpc annulus, interpolated across ICL fraction and redshift space, for a cluster with $\Mhalo=10^{14.7} $\Msun$$. $\fICL$ is defined as the total flux of the ICL across its entire radial extent, divided by the combined total fluxes of the ICL, BCG and cluster galaxies. Circles indicate the locations at which the ICL S/N has been measured to produce the interpolated map. (*fig:mhalo_z*)

</div>
<div id="div_fig2">

<img src="tmp_2503.17455/./ICL_fits_comparison_sigmaratio_ficl.png" alt="Fig11" width="100%"/>

**Figure 11. -** Ratio of the $\ICLap$  uncertainty $\referee${($\sigma_\mathrm{measured}$)} of each fitting method to the $\ICLap$  noise ($\referee${$\sigma_\mathrm{in}$, equivalent to the noise component in our S/N measurements)} of the input model, for varying $\fICL$ at a fixed halo mass of $\Mhalo=10^{14.7} $\Msun$$. Points are coloured by redshift. The horizontal lines show the mean value at each redshift, coloured by the corresponding redshift. (*fig:iclfits_comparison_sigma_ficl*)

</div>
<div id="div_fig3">

<img src="tmp_2503.17455/./sn_curve_multi_newsys_new.png" alt="Fig1" width="100%"/>

**Figure 1. -** S/N per kpc annulus versus radius for clusters of varying halo mass (_top_), ICL effective radius (_middle_) and ICL Sérsic index (_bottom_), measured on the isolated ICL component (not including BCG) within 2 kpc annuli. The lines and shaded regions respectively show the median value and standard deviation of the S/N across nine instances of the same cluster simulated in different regions of the image. $\referee${The vertical dotted lines indicate the 50--200 kpc region we selected for the annulus}.
     (*fig:sn-curve-multi*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.17455"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{ km s^{-1}}$
$\newcommand{\bibtex}{\textsc{Bib}\!\TeX}$
$\newcommand$
$\newcommand{\ngal}{ 17 }$
$\newcommand{\UWyoming}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\UAntof}{\affiliation{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\NOIRLab}{\affiliation{Gemini Observatory/NSF NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA}}$
$\newcommand{\UToledo}{\affiliation{Ritter Astrophysical Research Center, University of Toledo, Toledo, OH 43606, USA}}$
$\newcommand{\JHU}{\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218 USA}}$
$\newcommand{\Caltech}{\affiliation{TAPIR, California Institute of Technology, Pasadena, CA 91125 USA}}$
$\newcommand{\OSU}{\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave., Columbus, OH 43210, USA}}$
$\newcommand{\OSUphys}{\affiliation{Department of Physics, The Ohio State University, Columbus, Ohio 43210, USA}}$
$\newcommand{\OSUcosmo}{\affiliation{Center for Cosmology \& Astro-Particle Physics, The Ohio State University, Columbus, Ohio 43210, USA}}$
$\newcommand{\UMichigan}{\affiliation{Department of Astronomy, University of Michigan, Ann Arbor, MI 48109, USA}}$
$\newcommand{\MPIA}{\affiliation{Max Planck Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\ESO}{\affiliation{European Southern Observatory,  Karl-Schwarzschild Str. 2, 85748 Garching bei Muenchen, Germany}}$
$\newcommand{\UAlberta}{\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\UCSD}{\affiliation{Center for Astrophysics \& Space Sciences, Department of Physics,  University of California San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{\Belgium}{\affiliation{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\UHeidelberg}{\affiliation{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\Bonn}{\affiliation{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121, Bonn, Germany}}$
$\newcommand{\ANU}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\AthreeD}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\nrao}{\affiliation{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}}$
$\newcommand{\ITA}{\affiliation{Institut für Theoretische Astrophysik, Zentrum für Astronomie der Universität Heidelberg,\ Albert-Ueberle-Strasse 2, 69120 Heidelberg, Germany}}$
$\newcommand{\COOL}{\affiliation{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}}$
$\newcommand{\IWR}{\affiliation{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\MPE}{\affiliation{Max-Planck-Institut für Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}}$
$\newcommand{\ICRAR}{\affiliation{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{\OCADU}{\affiliation{OCAD University, Toronto, Ontario, M5T 1W1, Canada}}$
$\newcommand{\UArizona}{\affiliation{Steward Observatory, University of Arizona, 933 N Cherry Ave,Tucson, AZ 85721, USA}}$
$\newcommand{\Riverside}{\affiliation{Department of Physics and Astronomy, University of California, Riverside, CA, 92521 USA}}$
$\newcommand{\Connecticut}{\affiliation{University of Connecticut, Department of Physics, 196A  Auditorium Road, Unit 3046, Storrs, CT, 06269}}$
$\newcommand{\ARI}{\affiliation{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}}$
$\newcommand{\LaPlata}{\affiliation{Instituto de Astrofisica de La Plata, CONICET–UNLP,$
$ Paseo del Bosque S/N, B1900FWA La Plata, Argentina }}$
$\newcommand{\sorbonne}{\affiliation{Sorbonne {Université}, LERMA, Observatoire de Paris, PSL university, CNRS, F-75014, Paris, France}}$
$\newcommand{\CITA}{\affiliation{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}}$
$\newcommand{\MCMASTER}{\affiliation{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{\UNAM}{\affiliation{Instituto de Astronomía, Universidad Nacional Autónoma de México, Unidad Académica en Ensenada, Km 103 Carr. Tijuana−Ensenada, Ensenada, B.C.,$
$C.P. 22860, México}}$
$\newcommand{\Whitman}{\affil{Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}}$
$\newcommand{\ARC}{\affil{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\stsciesa}{\affil{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\oxford}{\affil{Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}}$
$\newcommand{\Carnegie}{\affil{The Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{\Virginia}{\affil{University of Virginia, Charlottesville, VA, USA }}$
$\newcommand{\Rechen}{\affil{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstraße 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\UniCA}{\affil{Université Côte d'Azur, Observatoire de la Côte d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}}$
$\newcommand{\Alberta}{\affil{Dept. of Physics, University of Alberta, 4-183 CCIS, Edmonton, Alberta, T6G 2E1, Canada}}$
$\newcommand{\Gent}{\affil{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\lea}{\mathrel{<\kern-1.0em\lower0.9ex\hbox{\sim}}}$
$\newcommand{\gea}{\mathrel{>\kern-1.0em\lower0.9ex\hbox{\sim}}}$</div>



<div id="title">

# Empirical SED Templates for Star Clusters Observed with HST and JWST: No Strong PAH or IR Dust Emission after Five Myr

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.17921-b31b1b.svg)](https://arxiv.org/abs/2503.17921)<mark>Appeared on: 2025-03-25</mark> -  _35 pages, 15 figures, 7 tables_

</div>
<div id="authors">

Bradley~C.~Whitmore, et al.

</div>
<div id="abstract">

**Abstract:** JWST observations, when combined withHST data, promise to improve age estimates of star clusters in nearby spiral galaxies.However, feedback from young cluster stars pushes out the natal gas and dust,making cluster formation and evolution a challenge to model.Here, we use JWST $+$ HST observations of the nearby spiral galaxy NGC 628 to producespectral energy distribution (SED) templates of compact star clustersspanning275 nm through 21 $\mu$ m. These preliminary SEDs capture the cluster stars and associated gas and dust within radii of $\approx0.12^{\prime\prime}$ to $0.67^{\prime\prime}$ (corresponding to $\approx6$ to $33$ pc at the distance of NGC 628).One important finding is that the SEDs of 1, 2, 3, and 4 Myr clusters can be differentiated in the infrared.  Another is that in 80-90 \% of the cases we study, the PAH and H $_{\alpha}$ emission track one another, with the dust responsible for the 3.3 $\micron$ PAH emission largely removed  by  4 Myr, consistent with pre-supernova stellar feedback acting quickly on the surrounding gas and dust.  Nearly-embedded cluster candidates have infrared SEDs which are quite similar to optically visible 1 to 3 Myr clusters. In nearly all cases we find there is a young star cluster within a few tenths of an arcsec (10 - 30 pc) of the nearly embedded cluster, suggesting  the formation of the cluster was triggered by its presence. The resulting age estimates from the empirical templates are compatible both with dynamical estimates based on CO superbubble expansion velocities, and the TODDLERS models which track spherical evolution of homogeneous gas clouds around young stellar clusters.

</div>

<div id="div_fig1">

<img src="tmp_2503.17921/./pilot4_1_4_mos_oct_25_2024.png" alt="Fig7" width="100%"/>

**Figure 7. -** Snapshots of  the 1 - 4 Myr training set clusters.  The cluster ID is shown in  yellow, along with the adopted age (see text). The color combinations  used for the snapshots, and a size scale, are shown in the top row. The program cluster is just above the small red cross in all cases.  (*fig:1_4_templates*)

</div>
<div id="div_fig2">

<img src="tmp_2503.17921/./pilot4_template_mosaic_oct_19_2024.png" alt="Fig9" width="100%"/>

**Figure 9. -** Figure showing how the mean-value version of the  empirical templates (upper right panel) are derived from the observations of the training set (upper left panel). The dashed lines are from the BC03 stellar continuum models. The black triangles are the 1 sigma detection  limits. The individual  clusters are shown in the bottom panel for the five clusters in each sample, although in some cases there are fewer data points because a cluster is an outlier (see Section \ref{sec:outliers}), or is off the bottom of the diagram. (*fig:mean_templates*)

</div>
<div id="div_fig3">

<img src="tmp_2503.17921/./pilot4_5_500_mos_oct_25_2024.png" alt="Fig8" width="100%"/>

**Figure 8. -** Same as Figure \ref{fig:1_4_templates} for 5 - 500 Myr training set clusters.   (*fig:5_500_templates*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.17921"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\xx}[1]{\textcolor{black}{ #1}}$
$\newcommand{\HII}{\ion{H}{II}}$
$\newcommand{\Ha}{H{\alpha}}$
$\newcommand{\ngal}{38}$
$\newcommand{\ngalphangs}{74}$
$\newcommand{\sfr}{M_{\odot} yr^{-1}}$
$\newcommand{\lea}{{\>\rlap{\raise2pt\hbox{<}}\lower3pt\hbox{\sim} \>}}$
$\newcommand{\gea}{{\>\rlap{\raise2pt\hbox{>}}\lower3pt\hbox{\sim} \>}}$
$\newcommand{\GEMINI}{\affiliation{Gemini Observatory/NSF’s NOIRLab, 950 N. Cherry Avenue, Tucson, AZ, 85719, USA$
$}}$
$\newcommand{\ASCL}{\affiliation{Astrophysics Source Code Librar$
$y, Michigan Technological University, 1400 Townsend Drive, Houghton, MI 49931}}$
$\newcommand{\OSU}{\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{\Alberta}{\affiliation{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\ANU}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\IPARCOS}{\affiliation{Instituto de Física de Partículas y del Cosmos, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{\IPAC}{\affiliation{Caltech-IPAC, 1200 E. California Blvd. Pasadena, CA 91125, USA}}$
$\newcommand{\Carnegie}{\affiliation{Observatories of the Carnegie Institution for Science, 813 Santa Barbara Street, Pasadena, CA 91101, USA}}$
$\newcommand{ÇAPP}{\affiliation{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\CfA}{\affiliation{Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{\CITEVA}{\affiliation{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\CNRS}{\affiliation{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\ESO}{\affiliation{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}}$
$\newcommand{\Heidelberg}{\affiliation{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\ICRAR}{\affiliation{International Centre for Radio Astronomy Research, University of Western Australia, 35 Stirling Highway, Crawley, WA 6009, Australia}}$
$\newcommand{\IRAM}{\affiliation{Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d'Hères, France}}$
$\newcommand{\IRAP}{\affiliation{CNRS, IRAP, 9 Av. du Colonel Roche, BP 44346, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UPS}{\affiliation{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\ITA}{\affiliation{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{\IWR}{\affiliation{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\JHU}{\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}}$
$\newcommand{\Leiden}{\affiliation{Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands}}$
$\newcommand{\Maryland}{\affiliation{Department of Astronomy, University of Maryland, College Park, MD 20742, USA}}$
$\newcommand{\MPE}{\affiliation{Max-Planck-Institut für extraterrestrische Physik, Giessenbachstra{\ss}e 1, D-85748 Garching, Germany}}$
$\newcommand{\MPIA}{\affiliation{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}}$
$\newcommand{\Nagoya}{\affiliation{Department of Physics, Nagoya University, Furo-cho, Chikusa-ku, Nagoya, Aichi 464-8602, Japan}}$
$\newcommand{\NRAO}{\affiliation{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{\OAN}{\affiliation{Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}}$
$\newcommand{\OCAD}{\affiliation{OCAD University, Toronto, Ontario, M5T 1W1, Canada}}$
$\newcommand{\ObsParis}{\affiliation{Sorbonne Université, Observatoire de Paris, Université PSL, CNRS, LERMA, F-75014, Paris, France}}$
$\newcommand{\Ox}{\affiliation{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}}$
$\newcommand{\Princeton}{\affiliation{Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544 USA}}$
$\newcommand{\UToledo}{\affiliation{Ritter Astrophysical Research Center, University of Toledo, Toledo, OH, 43606}}$
$\newcommand{\Toulouse}{\affiliation{Université de Toulouse, UPS-OMP, IRAP, F-31028 Toulouse cedex 4, France}}$
$\newcommand{\UBonn}{\affiliation{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}}$
$\newcommand{\UChile}{\affiliation{Departamento de Astronomía, Universidad de Chile, Camino del Observatorio 1515, Las Condes, Santiago, Chile}}$
$\newcommand{\UCM}{\affiliation{Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040 Madrid, Spain}}$
$\newcommand{\UCSD}{\affiliation{Center for Astrophysics and Space Sciences, Department of Physics,  University of California,\ San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{\ULyon}{\affiliation{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}}$
$\newcommand{\UMass}{\affiliation{University of Massachusetts—Amherst, 710 N. Pleasant Street, Amherst, MA 01003, USA}}$
$\newcommand{\UniCA}{\affiliation{Université Côte d'Azur, Observatoire de la Côte d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}}$
$\newcommand{\UWyoming}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\LAM}{\affiliation{$
$Aix Marseille Univ, CNRS, CNES, LAM (Laboratoire d’Astrophysique de Marseille),  F-13388 Marseille,$
$France}}$
$\newcommand{\UHawaii}{\affiliation{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}}$
$\newcommand{\UGent}{\affiliation{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}}$
$\newcommand{\IPARC}{\affiliation{Instituto de Física de Partículas y del Cosmos IPARCOS, Facultad de Ciencias Físicas, Universidad Complutense de Madrid, E-28040, Spain}}$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\McMaster}{\affiliation{Department of Physics and Astronomy, McMaster University, Hamilton, ON L8S 4M1, Canada}}$
$\newcommand{\INAF}{\affiliation{INAF -- Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50157, Firenze, Italy}}$
$\newcommand{\Sydney}{\affiliation{Sydney Institute for Astronomy, School of Physics A28, The University of Sydney, NSW 2006, Australia}}$
$\newcommand{\UA}{\affiliation{Centro de Astronomía (CITEVA), Universidad de Antofagasta, Avenida Angamos 601, Antofagasta, Chile}}$
$\newcommand{\LERMA}{\affiliation{Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universités, 75014 Paris}}$
$\newcommand{\SAIMSU}{\affiliation{Sternberg Astronomical Institute, Lomonosov Moscow State University, Universitetsky pr. 13, 119234 Moscow, Russia}}$
$\newcommand{\Rad}{\affiliation{Elizabeth S. and Richard M. Cashin Fellow at the Radcliffe Institute for Advanced Studies at Harvard University, 10 Garden Street, Cambridge, MA 02138, U.S.A.}}$
$\newcommand\kms{{\rm km s^{-1}}}$
$\newcommand\Kkms{{\rm K km s^{-1}}}$
$\newcommand\msun{{\mathrm{M}\_\odot}}$
$\newcommand\lsun{{\mathrm{L}\_\odot}}$
$\newcommand\micron{\hbox{\mum}}$
$\newcommand\solyr{{\rm M\_\odot yr^{-1}}}$
$\newcommand\sol{{\rm M\_\odot}}$
$\newcommand\dyncm{{\rm dyn cm^{-2}}}$
$\newcommand\ergs{{\rm  erg s^{-1}}}$
$\newcommand\cmsq{{\rm cm^{-2}}}$
$\newcommand\cmcb{{\rm cm^{-3}}}$
$\newcommand\gcmsq{{\rm g cm^{-2}}}$
$\newcommand\gcmcb{{\rm g cm^{-3}}}$
$\newcommand\Kcmcb{{\rm K cm^{-3}}}$
$\newcommand\ntdp{{\rm N\_2D^+}}$
$\newcommand\fluxdensityunitpix{{\rm erg s^{-1} cm^{-2} Å^{-1} pix^{-1}}}$
$\newcommand\fluxunitpix{{\rm erg s^{-1} cm^{-2} pix^{-1}}}$
$\newcommand\fluxdensityunit{{\rm erg s^{-1} cm^{-2} Å^{-1}}}$
$\newcommand\fluxunit{{\rm erg s^{-1} cm^{-2}}}$</div>



<div id="title">

# The PHANGS-HST-H$\alpha$ Survey: \ Warm Ionized Gas Physics at High Angular resolution in Nearby GalaxieS with the Hubble Space Telescope

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.18791-b31b1b.svg)](https://arxiv.org/abs/2503.18791)<mark>Appeared on: 2025-03-25</mark> -  _published in the Astronomical Journal_

</div>
<div id="authors">

R. Chandar, et al. -- incl., <mark>K. Kreckel</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** The PHANGS project is assembling a comprehensive, multi-wavelength dataset of nearby ( $\sim$ 5-20 Mpc), massive star-forming galaxies to enable multi-phase, multi-scale investigations into the processes that drive star formation and galaxy evolution. To date, large survey programs have provided molecular gas (CO) cubes with ALMA, optical IFU spectroscopy with VLT/MUSE, high-resolution NUV--optical imaging in five broad-band filters with HST, and infrared imaging in NIRCAM $+$ MIRI filters with JWST.  Here, we present PHANGS-HST-H $\alpha$ , which has obtained high-resolution ( $\sim$ 2 -- 10 pc), narrow-band imaging in the F658N or F657N filters with the HST/WFC3 camera of the warm ionized gas in the first 19 nearby galaxies observed in common by all four of the PHANGS large programs. We summarize our data reduction process, with a detailed discussion of the production of flux-calibrated, Milky Way extinction corrected, continuum-subtracted H $\alpha$ maps. PHANGS-MUSE IFU spectroscopy data are used to background subtract the HST-H $\alpha$ maps, and to determine the [ NII ] correction factors for each galaxy. We describe our public data products $\footnote{The data released as part of this work includes the reduced drizzled narrow-band images and the flux-calibrated, continuum-subtracted H$\alpha$ maps for each galaxy.  These images are available for download via MAST at \url{https://archive.stsci.edu/hlsp/phangs.html}, as well as at the Canadian Astronomy Data Centre as part of the PHANGS archive at \url{https://www.canfar.net/storage/vault/list/phangs/RELEASES}.}$ and highlight a few key science cases enabled by the PHANGS-HST-H $\alpha$ observations.

</div>

<div id="div_fig1">

<img src="tmp_2503.18791/./figures/ngc628c_scatter_anchor_contsub.png" alt="Fig1" width="100%"/>

**Figure 1. -** **Results of the [N II] $\lambda$6548Å and [N II] $\lambda$6583Å line correction routine (\S \ref{subsec_contsub2**).} H$\alpha$ fluxes from the continuum subtracted MUSE synthetic filter maps as a function of the H$\alpha$ flux densities measured from the MUSE spectroscopic data. The blue circles represent 20 binned data points (equally spaced), and the dashed blue line shows the best linear fit to these points ($y = ax + b$), where $y$ represents the HST image data points and $x$ represents the MUSE data points (see Table \ref{tab_bgcorr_niicorr}). The dotted line shows the 1-to-1 line. (*fig_scatteranchor*)

</div>
<div id="div_fig2">

<img src="tmp_2503.18791/./ms_ks.png" alt="Fig6" width="100%"/>

**Figure 6. -** ** Left:** Galaxies from the SDSS sample with $z \lesssim 0.1$ form the backdrop of the left panel, where the galaxy star-forming sequence is the prominent upper feature. Coverage of the SFR--M$_*$ plane by galaxies in the parent PHANGS sample ($N=74$; open circles) is shown.
        Red crosses show that the 19 galaxies observed as part of the HST-H$\alpha$ project tend to have higher SFRs.
        ** Right:** The star formation and molecular gas surface brightness  ([Sun, Leroy and Rosolowsky 2022]())  for the parent PHANGS sample (open circles) and the PHANGS-HST-H$\alpha$ subsample studied here (red crosses).  Outlier NGC 3239 is noted.
     (*fig:sample*)

</div>
<div id="div_fig3">

<img src="tmp_2503.18791/./figures/ngc628c_filters_spec.png" alt="Fig9.1" width="50%"/><img src="tmp_2503.18791/./figures/ngc2835s_filters_spec.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** **Example MUSE spectra compared with HST filter transmission curves (ACS/WFC and WFC3/UVIS).**
    Example spectra from NGC 628 (top panel) and NGC 2835 (bottom panel) are shown as solid black lines, overlaid with the F555W (blue), F658N (orange), and F814W (green) filter transmission curves for ACS/WFC (top panel) and WFC3/UVIS  (bottom panel). The left panel displays the full wavelength range of the MUSE observations, with dashed vertical lines marking the coverage limits. In the right panel, we focus on the $\Ha$ line within the F658N filter bandpass, which includes contributions from the [NII]$\lambda6548$ and [NII]$\lambda6583$ emission lines. (*fig_filters_spec*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.18791"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

406  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

41  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

10  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
